## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-07-04-00-00 +0000,wsj,See How a Chinese Invasion of Taiwan Could Unfold,https://www.wsj.com/world/china/how-china-chin...
1,2025-12-07-03-53-43 +0000,nypost,Boston Archdiocese calls for removal of ‘ICE w...,https://nypost.com/2025/12/06/us-news/boston-a...
2,2025-12-07-03-46-56 +0000,nypost,Warner Bros. Discovery CEO’s bidding war destr...,https://nypost.com/2025/12/06/business/warner-...
3,2025-12-07-03-45-33 +0000,nyt,Survivors of the Deadly Hong Kong Fire Are in ...,https://www.nytimes.com/2025/12/06/world/asia/...
4,2025-12-07-03-34-14 +0000,nypost,Three inmates escape from Louisiana jail — cop...,https://nypost.com/2025/12/06/us-news/three-in...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
89,trump,23
35,fire,8
93,new,7
170,up,7
16,bros,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
146,2025-12-06-05-14-15 +0000,nypost,White House slams Newsom after gov claims Trum...,https://nypost.com/2025/12/06/us-news/white-ho...,53
41,2025-12-06-20-49-19 +0000,nypost,Qatari leader warns Gaza cease-fire at ‘critic...,https://nypost.com/2025/12/06/world-news/qatar...,52
116,2025-12-06-11-45-00 +0000,nypost,"Zelensky, Trump admin have ‘constructive’ phon...",https://nypost.com/2025/12/06/world-news/russi...,50
62,2025-12-06-19-04-27 +0000,wapo,Trump urges a new vaccine schedule. Here’s wha...,https://www.washingtonpost.com/world/2025/12/0...,43
145,2025-12-06-06-19-39 +0000,nypost,Trump administration will expand travel ban to...,https://nypost.com/2025/12/06/us-news/trump-ad...,43


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
146,53,2025-12-06-05-14-15 +0000,nypost,White House slams Newsom after gov claims Trum...,https://nypost.com/2025/12/06/us-news/white-ho...
2,41,2025-12-07-03-46-56 +0000,nypost,Warner Bros. Discovery CEO’s bidding war destr...,https://nypost.com/2025/12/06/business/warner-...
52,38,2025-12-06-19-51-59 +0000,latimes,"Christmas Tree Lane will light up tonight, car...",https://www.latimes.com/california/story/2025-...
101,31,2025-12-06-14-35-09 +0000,nypost,Stephen Miller calls billion-dollar Minnesota ...,https://nypost.com/2025/12/06/us-news/stephen-...
148,30,2025-12-06-04-27-00 +0000,wsj,A key vaccine panel at the Centers for Disease...,https://www.wsj.com/politics/policy/cdc-panel-...
71,30,2025-12-06-18-10-02 +0000,nypost,Deadly Hong Kong fire came after residents wer...,https://nypost.com/2025/12/06/world-news/hong-...
29,30,2025-12-06-23-09-00 +0000,wsj,Defense Secretary Pete Hegseth defended the U....,https://www.wsj.com/politics/national-security...
63,23,2025-12-06-18-58-09 +0000,bbc,Can you ban kids from social media? Australia ...,https://www.bbc.com/news/articles/crk7xgzj8y8o...
80,23,2025-12-06-17-02-19 +0000,nypost,Mayor Adams’ one-time roomie stepping down fro...,https://nypost.com/2025/12/06/us-news/mayors-f...
140,23,2025-12-06-09-00-58 +0000,nypost,DHS fires back after Dem Rep. Adelita Grijalva...,https://nypost.com/2025/12/06/us-news/rep-adel...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
